In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Cristóvão Beirão da Cruz e Silva <cbeiraod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

#############################################################################
# Modified for ETROC2 TID Plotting, Zachary Flowers <zachary.flowers@cern.ch>
#############################################################################

## Imports

In [ ]:
from glob import glob
from natsort import natsorted
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use('CMS')
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os, sys
import datetime
from fnmatch import fnmatch
import scipy.stats as stats
from math import ceil
from numpy import savetxt
import pickle
from cycler import cycler

### Line to copy data from EOS to local

#### Note: need to 'kinit' first
May not work from notebook
Might be best to run from terminal

In [ ]:
os.system('mkdir -p /home/daq/ETROC2/ETROC-Data/cern_Jan2024/ && xrdcp -r root://cmseos.fnal.gov///store/group/lpcmtdstudies/cern_Jan2024/TID /home/daq/ETROC2/ETROC-Data/cern_Jan2024/')

# Analysis

In [ ]:
def multiple_trigger_bit_noisescan_plot(root, file_pattern, chip_figtitle, chip_figname, scan_list, attempts=[], tags=[], colors = ['k'], slide_friendly=False):
    scan_name = chip_figname+"_VRef_SCurve_NoiseOnly"
    triggerbit_full_Scurve = {row:{col:{attempt:{} for attempt in attempts} for col in range(16)} for row in range(16)}

    today = datetime.date.today().isoformat()

    fig_outdir = Path('../../ETROC-figures')
    fig_outdir = fig_outdir / (today + '_Array_Test_Results')
    fig_outdir.mkdir(exist_ok=True)
    fig_path = str(fig_outdir)

    row_list, col_list = zip(*scan_list)
    u_cl = np.sort(np.unique(col_list))
    u_rl = np.sort(np.unique(row_list))
    
    for row,col in scan_list:
        for attempt in attempts:
            path_pattern = f"*_Array_Test_Results*/{scan_name}_Pixel_C{col}_R{row}"+attempt
            file_list = []
            for path, subdirs, files in os.walk(root):
                if not fnmatch(path, path_pattern): continue
                for name in files:
                    pass
                    if fnmatch(name, file_pattern):
                        file_list.append(os.path.join(path, name))
            # print(file_list)
            for file_index, file_name in enumerate(file_list):
                with open(file_name) as infile:
                    for line in infile:
                        text_list = line.split(',')
                        FPGA_triggerbit = int(text_list[5])
                        DAC = int(text_list[-1])
                        if DAC == -1: continue
                        triggerbit_full_Scurve[row][col][attempt][DAC] = FPGA_triggerbit

    if slide_friendly:
        fig = plt.figure(dpi=100, figsize=(len(np.unique(u_cl))*10,len(np.unique(u_rl))*9))
        gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
    for ri,row in enumerate(u_rl):
        for ci,col in enumerate(u_cl):
            x_max = 0
            x_min = 1000
            if slide_friendly:
                ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
            else:
                fig, ax0 = plt.subplots(dpi=100, figsize=(10,9))
            for attempt, tag, color in zip(attempts, tags, colors):
                Y = np.array(list(triggerbit_full_Scurve[row][col][attempt].values()))
                X = np.array(list(triggerbit_full_Scurve[row][col][attempt].keys()))
                ax0.plot(X, Y, '.-', color=color,lw=1.0,label=tag)
                ax0.set_xlabel("DAC Value [decimal]")
                ax0.set_ylabel("Trigger Bit Counts [decimal]")
                hep.cms.text(loc=0, ax=ax0, text="Phase-2 Preliminary", fontsize=15)
                min_x_point = X[np.argmin(Y)]
                max_x_point = X[np.argmax(Y)]
                if(max_x_point > x_max):
                    x_max = max_x_point
                if(min_x_point < x_min):
                    x_min = min_x_point
            ax0.set_xlim(left=x_min-x_min*0.02, right=x_max+x_max*0.02)
            plt.yscale("log")
            plt.title(f"{chip_figtitle}, Pixel ({row},{col}) "+tag,size=15, loc="right")
            plt.tight_layout()
            # plt.legend(loc="upper right", fontsize=14)
            plt.legend(loc='best')
            if not slide_friendly:
                plt.savefig(fig_path+"/"+chip_figname+"_NoisePeak_Log"+"_Pixel_R"+str(row)+"_C"+str(col)+attempt+".png")
                plt.savefig(fig_path+"/"+chip_figname+"_NoisePeak_Log"+"_Pixel_R"+str(row)+"_C"+str(col)+attempt+".pdf")
                plt.close()
    if slide_friendly:
        plt.savefig(fig_path+"/"+chip_figname+"_NoisePeak_Log"+attempts[0]+"_multiple.png")
        plt.savefig(fig_path+"/"+chip_figname+"_NoisePeak_Log"+attempts[0]+"_multiple.pdf")
        plt.close()

    if slide_friendly:
        fig = plt.figure(dpi=100, figsize=(len(np.unique(u_cl))*10,len(np.unique(u_rl))*9))
        gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
    for ri,row in enumerate(u_rl):
        for ci,col in enumerate(u_cl):
            x_max = 0
            x_min = 1000
            if slide_friendly:
                ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
            else:
                fig, ax0 = plt.subplots(dpi=100, figsize=(10,9))
            for attempt, tag, color in zip(attempts, tags, colors):
                Y = np.array(list(triggerbit_full_Scurve[row][col][attempt].values()))
                X = np.array(list(triggerbit_full_Scurve[row][col][attempt].keys()))
                ax0.plot(X, Y, '.-', color=color,lw=1.0,label=tag)
                ax0.set_xlabel("DAC Value [decimal]")
                ax0.set_ylabel("Trigger Bit Counts [decimal]")
                hep.cms.text(loc=0, ax=ax0, text="Phase-2 Preliminary", fontsize=15)
                min_x_point = X[np.argmin(Y)]
                max_x_point = X[np.argmax(Y)]
                if(max_x_point > x_max):
                    x_max = max_x_point
                if(min_x_point < x_min):
                    x_min = min_x_point
            ax0.set_xlim(left=x_min-x_min*0.02, right=x_max+x_max*0.02)
            plt.yscale("linear")
            plt.ticklabel_format(useOffset=False,axis='both',style='plain')
            plt.title(f"{chip_figtitle}, Pixel ({row},{col}) "+tag,size=15, loc="right")
            plt.tight_layout()
            # plt.legend(loc="upper right", fontsize=14)
            plt.legend(loc='best')
            if not slide_friendly:
                plt.savefig(fig_path+"/"+chip_figname+"_NoisePeak_Linear"+"_Pixel_R"+str(row)+"_C"+str(col)+attempt+".png")
                plt.savefig(fig_path+"/"+chip_figname+"_NoisePeak_Linear"+"_Pixel_R"+str(row)+"_C"+str(col)+attempt+".pdf")
                plt.close()
    if slide_friendly:
        plt.savefig(fig_path+"/"+chip_figname+"_NoisePeak_Linear"+attempts[0]+"_multiple.png")
        plt.savefig(fig_path+"/"+chip_figname+"_NoisePeak_Linear"+attempts[0]+"_multiple.pdf")
        plt.close()
    del triggerbit_full_Scurve

In [ ]:
root = '/home/daq/ETROC2/ETROC-Data/cern_Jan2024/TID/'
file_pattern = "*FPGA_Data.dat"
chip_figtitle = "ET2.01 W36-6"
chip_figname = "ET2p01_W36_6"
row_list = [0, 0, 3, 3, 12, 12]
col_list = [14, 2, 14, 2, 14, 2]
scan_list = list(zip(row_list, col_list))
attempts = ["_m25c_0MRad","_m25c_200MRad","_m25c_400MRad"]
tags = ["12C, 0, MRad","-25C, 200, MRad","-25C, 400, MRad"]
colors = ['k','r','b']
multiple_trigger_bit_noisescan_plot(root,file_pattern,chip_figtitle,chip_figname,scan_list,attempts,tags,colors,False)

## QInj vs DAC

In [ ]:
def load_auto_cal_BL_map(chip_name, user_path="", day=datetime.date.today().isoformat()):
    indir = Path('/home/daq/ETROC2/ETROC-Data/cern_Jan2024/TID/'+(day+'_Array_Test_Results/')+user_path)
    infile_BL_map = indir / (chip_name+"_BL_map.pickle")
    with open(infile_BL_map, 'rb') as f:
        return pickle.load(f)

def load_auto_cal_NW_map(chip_name, user_path="", day=datetime.date.today().isoformat()):
    indir = Path('/home/daq/ETROC2/ETROC-Data/cern_Jan2024/TID/'+(day+'_Array_Test_Results/')+user_path)
    infile_NW_map = indir / (chip_name+"_NW_map.pickle")
    with open(infile_NW_map, 'rb') as f:
        return pickle.load(f)

In [ ]:
def charge_peakDAC_plot(chip_name, chip_figtitle, chip_figname, scan_list, QInjEns, attempt='', tag='', day = '', pkl_day = '', plot_BL=False, do_Fit=True, rm_n_fit_points=0, slide_friendly=False):
    scan_name = chip_figname+"_VRef_SCurve_BinarySearch_TurnOff"
    if(plot_BL):
        BL_map_THCal=load_auto_cal_BL_map(chip_name, "", pkl_day)
        NW_map_THCal=load_auto_cal_NW_map(chip_name, "", pkl_day)
    QInj_Peak_DAC_map = {row:{col:{q:0 for q in QInjEns} for col in range(16)} for row in range(16)}

    today = datetime.date.today().isoformat()

    fig_outdir = Path('../../ETROC-figures')
    fig_outdir = fig_outdir / (today + '_Array_Test_Results')
    fig_outdir.mkdir(exist_ok=True)
    fig_path = str(fig_outdir)

    for row,col in scan_list:
        for QInj in QInjEns:
            threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}'
            path_pattern = f"*{day}_Array_Test_Results/{threshold_name}"
            file_list = []
            for path, subdirs, files in os.walk(root):
                if not fnmatch(path, path_pattern): continue
                for name in files:
                    pass
                    if fnmatch(name, file_pattern):
                        file_list.append(os.path.join(path, name))
            for file_index, file_name in enumerate(file_list):
                with open(file_name) as infile:
                    last_line = infile.readlines()[-1]
                    text_list = last_line.split(',')
                    DAC = int(text_list[-1])
                    QInj_Peak_DAC_map[row][col][QInj] = DAC

    row_list, col_list = zip(*scan_list)
    u_cl = np.sort(np.unique(col_list))
    u_rl = np.sort(np.unique(row_list))
    if slide_friendly:
        fig = plt.figure(dpi=100, figsize=(len(np.unique(u_cl))*10,len(np.unique(u_rl))*9))
        gs = fig.add_gridspec(len(np.unique(u_rl)),len(np.unique(u_cl)))
    for ri,row in enumerate(u_rl):
        for ci,col in enumerate(u_cl):
            if(slide_friendly):
                ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
            else:
                fig, ax0 = plt.subplots(dpi=100, figsize=(10,9))
            if(plot_BL):
                BL = int(np.floor(BL_map_THCal[row][col]))
                NW = abs(int(np.floor(NW_map_THCal[row][col])))
                ax0.axhline(BL, color='k', lw=0.8, label=f"BL = {BL} DAC LSB")
                ax0.axhline(BL+NW, color='k',ls="--", lw=0.8, label=f"NW = $\pm${NW} DAC LSB")
                ax0.axhline(BL-NW, color='k',ls="--", lw=0.8)
            X = []
            Y = []
            for QInj in QInjEns:
                ax0.plot(QInj, QInj_Peak_DAC_map[row][col][QInj], 'rx')
                X.append(QInj)
                Y.append(QInj_Peak_DAC_map[row][col][QInj])
            X = np.array(X[rm_n_fit_points:])
            Y = np.array(Y[rm_n_fit_points:])
            if(do_Fit):
                (m, b), cov = np.polyfit(X, Y, 1, cov = True)
                n = Y.size
                Yfit = np.polyval((m,b), X)
                errorbars = np.sqrt(np.diag(cov))
                x_range = np.linspace(0, 35, 100)
                y_est = b + m*x_range
                resid = Y - Yfit
                s_err = np.sqrt(np.sum(resid**2)/(n - 2))
                t = stats.t.ppf(0.95, n - 2)
                ci2= t * s_err * np.sqrt(    1/n + (x_range - np.mean(X))**2/(np.sum((X)**2)-n*np.sum((np.mean(X))**2)))

            if(do_Fit):
                ax0.plot(x_range, y_est, 'b-', lw=-.8, label=f"DAC_TH = ({m:.2f}$\pm${errorbars[0]:.2f})$\cdot$Q + ({b:.2f}$\pm${errorbars[1]:.2f})")
                plt.fill_between(x_range, y_est+ci2, y_est-ci2, color='b',alpha=0.2, label="95% Confidence Interval on Linear Fit")
            ax0.set_xlabel("Charge Injected [fC]")
            ax0.set_ylabel("DAC Threshold [LSB]")
            hep.cms.text(loc=0, ax=ax0, text="Phase-2 Preliminary", fontsize=15)
            plt.title(f"{chip_figtitle}, Pixel ({row},{col})"+tag, size=15, loc='right')
            if(do_Fit or plot_BL):
                # plt.legend(loc=(0.04,0.65))
                plt.legend(loc='best')
            plt.tight_layout()
            if not slide_friendly:
                plt.savefig(fig_path+"/"+chip_figname+"_QInj_Sensitivity"+"_Pixel_R"+str(row)+"_C"+str(col)+attempt+".png")
                plt.savefig(fig_path+"/"+chip_figname+"_QInj_Sensitivity"+"_Pixel_R"+str(row)+"_C"+str(col)+attempt+".pdf")
                plt.close()
    if slide_friendly:
        plt.savefig(fig_path+"/"+chip_figname+"_QInj_Sensitivity"+attempt+".png")
        plt.savefig(fig_path+"/"+chip_figname+"_QInj_Sensitivity"+attempt+".pdf")
        plt.close()
    del QInj_Peak_DAC_map

In [ ]:
charge_peakDAC_plot(chip_name="ET2p01_W36_6", chip_figtitle="ET2.01 W36-6, 12\N{DEGREE SIGN}C, 0 MRad", chip_figname="ET2p01_W36_6", scan_list=scan_list, QInjEns=[5, 6, 8, 15, 22, 27], attempt='_p12_0MRad', tag='', day = '2024-01-1*', pkl_day='2024-01-10', plot_BL=False, do_Fit=True, rm_n_fit_points=2, slide_friendly=False)
charge_peakDAC_plot(chip_name="ET2p01_W36_6", chip_figtitle="ET2.01 W36-6, 12\N{DEGREE SIGN}C, 0 MRad", chip_figname="ET2p01_W36_6", scan_list=scan_list, QInjEns=[5, 6, 8, 15, 22, 27], attempt='_p12_0MRad', tag='', day = '2024-01-1*', pkl_day='2024-01-11', plot_BL=False, do_Fit=True, rm_n_fit_points=2, slide_friendly=False)
charge_peakDAC_plot(chip_name="ET2p01_W36_6", chip_figtitle="ET2.01 W36-6, -25\N{DEGREE SIGN}C, 200 MRad", chip_figname="ET2p01_W36_6", scan_list=scan_list, QInjEns=[5, 6, 8, 15, 22, 27], attempt='_m25c_200MRad', tag='', day = '2024-01-1*', pkl_day='2024-01-15', plot_BL=False, do_Fit=True, rm_n_fit_points=2, slide_friendly=False)
charge_peakDAC_plot(chip_name="ET2p01_W36_6", chip_figtitle="ET2.01 W36-6, -25\N{DEGREE SIGN}C, 400 MRad", chip_figname="ET2p01_W36_6", scan_list=scan_list, QInjEns=[5, 6, 8, 15, 22, 27], attempt='_m25c_400MRad', tag='', day = '2024-01-1*', pkl_day='2024-01-15', plot_BL=False, do_Fit=True, rm_n_fit_points=2, slide_friendly=False)

# Money Plots

## Feather making (already done for CERN Jan 2024)

In [ ]:
def toSingleDataFramePerDirectory_newEventModel(
        path_to_dir: str,
        dir_name_pattern: str,
        save_to_csv: bool = False,
        debugging: bool = False,
        output_dir: str = "",
        extra_str: str = "",
    ):

    if output_dir != "":
        os.system(f"mkdir -p {output_dir}")
    name_pattern = "*translated*.nem"

    dirs = glob(f"{path_to_dir}/{dir_name_pattern}")
    dirs = natsorted(dirs)
    print(dirs[:3])

    if debugging:
        dirs = dirs[:1]

    d = {
        # 'board': [],
        'row': [],
        'col': [],
        'toa': [],
        'tot': [],
        'cal': [],
    }

    for dir in tqdm(dirs):
        df = pd.DataFrame(d)
        name = dir.split('/')[-1]
        files = glob(f"{dir}/{name_pattern}")

        for ifile in files:
            file_d = copy.deepcopy(d)

            if os.stat(ifile).st_size == 0:
                continue

            with open(ifile, 'r') as infile:
                for line in infile:
                    if line.split(' ')[0] == 'EH':
                        pass
                    elif line.split(' ')[0] == 'H':
                        pass
                        # bcid = int(line.split(' ')[-1])
                    elif line.split(' ')[0] == 'D':
                        # id  = int(line.split(' ')[1])
                        col = int(line.split(' ')[-4])
                        row = int(line.split(' ')[-5])
                        toa = int(line.split(' ')[-3])
                        tot = int(line.split(' ')[-2])
                        cal = int(line.split(' ')[-1])
                        # file_d['evt'].append(evt)
                        # file_d['board'].append(id)
                        file_d['row'].append(row)
                        file_d['col'].append(col)
                        file_d['toa'].append(toa)
                        file_d['tot'].append(tot)
                        file_d['cal'].append(cal)
                    elif line.split(' ')[0] == 'T':
                        pass
                    elif line.split(' ')[0] == 'ET':
                        pass
            # if len(file_d['evt']) > 0:
            file_df = pd.DataFrame(file_d)
            df = pd.concat((df, file_df), ignore_index=True)
            del file_df
            del file_d

        if not df.empty:
            df = df.astype('int')
            if save_to_csv:
                df.to_csv(name+'.csv', index=False)
            else:
                df.to_feather(f"{output_dir}/{name}{extra_str}.feather")
            del df

In [ ]:
toSingleDataFramePerDirectory_newEventModel(
    path_to_dir = '../../ETROC-Data/cern_Jan2024/TID/*PlateTemp12C_DryAir*/',
    dir_name_pattern = 'ET2p01_W36_6_VRef_SCurve_TDC_*',
    output_dir = '../../ETROC-Data/cern_Jan2024/TID/0MRad_Data/',
    extra_str = '_p12c_0MRad',
    # save_to_csv = True,
    # debugging = True
)

In [ ]:
files = natsorted(glob('../../ETROC-Data/cern_Jan2024/TID/400MRad_Data/*'))
files

In [ ]:
data_dict = {
    'row': [],
    'col': [],
    'charge': [],
    'dac': [],
    'temp': [],
    'dose': [],
    'toa_mean': [],
    'toa_std': [],
    'tot_mean': [],
    'tot_std': [],
    'cal_mean': [],
    'cal_std': [],
}

for ifile in tqdm(files):
    fname = ifile.split('/')[-1].split('.')[0]

    # row = fname.split('_')[-5].split('R')[1]
    # col = fname.split('_')[-6].split('C')[1]
    # charge = fname.split('_')[-3]
    # dac = fname.split('_')[-1]

    row = fname.split('_')[-7].split('R')[1]
    col = fname.split('_')[-8].split('C')[1]
    charge = fname.split('_')[-5]
    dac = fname.split('_')[-3]
    temp = fname.split('_')[-2][1:-1]
    if fname.split('_')[-2][0] == 'm':
        temp = '-'+temp
    dose = fname.split('_')[-1][0:-4]

    # print(row, col, charge, dac)

    df = pd.read_feather(ifile)
    df = df.loc[(df['cal'] >= df['cal'].mode()[0]-2) & (df['cal'] <= df['cal'].mode()[0]+2)]

    data_dict['row'].append(int(row))
    data_dict['col'].append(int(col))
    data_dict['charge'].append(int(charge))
    data_dict['dac'].append(int(dac))
    data_dict['temp'].append(int(temp))
    data_dict['dose'].append(int(dose))

    data_dict['toa_mean'].append(df['toa'].mean())
    data_dict['toa_std'].append(df['toa'].std())
    data_dict['tot_mean'].append(df['tot'].mean())
    data_dict['tot_std'].append(df['tot'].std())
    data_dict['cal_mean'].append(df['cal'].mean())
    data_dict['cal_std'].append(df['cal'].std())

In [ ]:
cut_df = pd.DataFrame(data=data_dict)

In [ ]:
final_df = cut_df.loc[(cut_df['toa_std'] <= 4)]

In [ ]:
final_df.to_feather('ET2p01_W36_6_TDC_summary_400MRad.feather')

## Analysis After Making Feather

### Setup Analysis

In [ ]:
feather_path = '/home/daq/ETROC2/ETROC-Data/cern_Jan2024/TID/'
final_df = pd.read_feather(feather_path+'ET2p01_W36_6_TDC_summary_allMRad.feather')
final_df

In [ ]:
def round_down_to_nearest_hundred(number):
    rounded_number = (number // 100) * 100
    return rounded_number

def round_down_to_nearest_ten(number):
    rounded_number = (number // 10) * 10
    return rounded_number

In [ ]:
def make_moneyplot(
        input_df: pd.DataFrame,
        pixel: list[set],
        value_name: str,
        board_name: str,
        title_tag: str,
        dose: int,
        ylabel: str,
    ):

    sub_df = input_df.loc[(input_df['row'] == pixel[0]) & (input_df['col'] == pixel[1]) & (input_df['dose'] == dose)]

    
    custom_cycler = (cycler(color=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02'])
                    + cycler(marker=['o', 'X', '^', 'D', 's', '*']))

    fig, ax = plt.subplots(figsize=(10, 9))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=15)
    ax.set_prop_cycle(custom_cycler)
    ax.grid()

    for icharge in input_df['charge'].unique():
        tmp_df = sub_df.loc[sub_df['charge'] == icharge]
        plt.plot(tmp_df['dac'], tmp_df[value_name], label=f"{icharge} fC")

    ax.set_title(f"{board_name} Pixel ({pixel[0]},{pixel[1]}) {title_tag}, {dose} MRad", size=15, loc="right")
    ax.set_xlim(round_down_to_nearest_ten(tmp_df['dac'].min())-10, None)
    # ax.tick_params(axis='x', which='major', pad=7)
    # ax.tick_params(axis='y', which='major', pad=8)
    # ax.set_ylim(-0.2, 8)
    ax.legend(loc='best')
    ax.set_xlabel('DAC value [LSB]')
    ax.set_ylabel(ylabel)

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
    dir = "../../ETROC-figures/"+datetime.date.today().isoformat() + '_Array_Test_Results/Normal_Money/'
    os.system(f'mkdir -p {dir}')
    plt.savefig(dir+board_name+"_R"+str(pixel[0])+"_C"+str(pixel[1])+"_TID_"+str(dose)+"_"+value_name+".png")
    plt.savefig(dir+board_name+"_R"+str(pixel[0])+"_C"+str(pixel[1])+"_TID_"+str(dose)+"_"+value_name+".pdf")
    plt.close()

In [ ]:
def make_TID_moneyplot(
        input_df: pd.DataFrame,
        pixel: list[set],
        value_name: str,
        board_name: str,
        title_tag: str,
        QInj: int,
        ylabel: str,
    ):

    sub_df = input_df.loc[(input_df['row'] == pixel[0]) & (input_df['col'] == pixel[1]) & (input_df['charge'] == QInj)]

    
    custom_cycler = (cycler(color=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02'])
                    + cycler(marker=['o', 'X', '^', 'D', 's', '*']))

    fig, ax = plt.subplots(figsize=(10, 9))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=15)
    ax.set_prop_cycle(custom_cycler)
    ax.grid()
    x_min = 1000
    for idose in input_df['dose'].unique():
        tmp_df = sub_df.loc[sub_df['dose'] == idose]
        temp = tmp_df['temp'].unique()[0]
        current_x_min = round_down_to_nearest_ten(tmp_df['dac'].min())-10
        if (current_x_min < x_min):
            x_min = current_x_min
        plt.plot(tmp_df['dac'], tmp_df[value_name], label=f"{idose} MRad {temp}\N{DEGREE SIGN}C")

    title = f"{board_name} Pixel ({pixel[0]},{pixel[1]}) {title_tag}"
    ax.set_title(f"{board_name} Pixel ({pixel[0]},{pixel[1]}) {title_tag}", size=15, loc="right")
    ax.set_xlim(x_min, None)
    # ax.tick_params(axis='x', which='major', pad=7)
    # ax.tick_params(axis='y', which='major', pad=8)
    # ax.set_ylim(-0.2, 8)
    if 'mean' in value_name:
        ax.legend(loc='best')
    else:
        ax.legend(loc='best')
    ax.set_xlabel('DAC value [LSB]')
    ax.set_ylabel(ylabel)

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
    dir = "../../ETROC-figures/"+datetime.date.today().isoformat() + '_Array_Test_Results/TID_Money/'
    os.system(f'mkdir -p {dir}')
    plt.savefig(dir+board_name+"_R"+str(pixel[0])+"_C"+str(pixel[1])+"_QInj_"+str(QInj)+"_TID_"+value_name+".png")
    plt.savefig(dir+board_name+"_R"+str(pixel[0])+"_C"+str(pixel[1])+"_QInj_"+str(QInj)+"_TID_"+value_name+".pdf")
    plt.close()

In [ ]:
pixels = [(0,2), (0,8), (0,14), (3,2), (3,8), (3,14), (12,2), (12,8), (12,14)]
doses = [0,200,400]
QInjEns = [5, 6, 8, 15, 22, 27]
value_names = ["toa_mean","toa_std","tot_mean","tot_std","cal_mean","cal_std"]
ylabels = ["TOA Mean","TOA Std","TOT Mean","TOT Std","CAL Mean","CAL Std"]
vals = list(zip(value_names,ylabels))

## Make ALL Money Plots

In [ ]:
for value_name, label in vals:
    for ipix in pixels:
        for dose in doses:
            make_moneyplot(final_df, ipix, value_name=value_name, board_name='ET2 W36-6', title_tag='QInj', dose=dose, ylabel=label+' [LSB]')

In [ ]:
for value_name, label in vals:
    for ipix in pixels:
        for QInj in QInjEns:
            make_TID_moneyplot(final_df, ipix, value_name=value_name, board_name='ET2 W36-6', title_tag='TID', QInj=QInj, ylabel=label+' [LSB]')

#### Only plot single dose if not desired to remake ALL plots

In [ ]:
pixels = [(0,2), (0,8), (0,14), (1,2), (1,8), (1,14), (3,2), (3,8), (3,14), (12,2), (12,8), (12,14)]

for ipix in pixels:
    make_moneyplot(final_df, ipix, value_name='toa_mean', board_name='ET2.01 W36-6', title_tag='QInj Money plot 200MRad',temperature='-25C', ylabel='TOA Mean [LSB]')